In [ ]:
import os
import time
import logging
import random
import json
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from concurrent.futures import ThreadPoolExecutor

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Path to Solana wallets
KEYS_FOLDER = './solana_wallets'

# Target URL
target_url = "https://pump.fun/HT66GuGkuE3ohh4hZnE2bbkiCeES9kP87h3w1qs29ekF"

# List of user agents to randomize the browser identities
USER_AGENTS = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15',
]

# Load Solana wallets
def load_wallets(dir: str):
    wallets = []
    for filename in os.listdir(dir):
        if filename.endswith(".json"):
            with open(os.path.join(dir, filename), 'r') as f:
                wallet = json.load(f)
                wallets.append(wallet)
    return wallets

wallets = load_wallets(KEYS_FOLDER)
wallet_index = 0

# Initialize the WebDriver for Selenium
def initialize_driver(user_agent):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument(f"user-agent={user_agent}")
    options.add_argument("--window-size=1920,1080")
    return webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# Function to extract specific cookies from a cookie string
def extract_cookies(cookie_string, names):
    cookies = {}
    for cookie in cookie_string.split(';'):
        name, value = cookie.split('=', 1)
        name = name.strip()
        value = value.strip()
        if name in names:
            cookies[name] = value
    return cookies

# Function to send a POST request to the chat API
def send_post_request(text, mint, token, auth, additional_cookies):
    cookie_string = f"auth_token={auth}"
    for name, value in additional_cookies.items():
        cookie_string += f"; {name}={value}"

    headers = {
        "Accept": "*/*",
        "Accept-Encoding": "gzip, deflate, br, zstd",
        "Accept-Language": "en-US,en;q=0.9,en-GB-oxendict;q=0.8,en-GB;q=0.7",
        "Content-Type": "application/json",
        "Cookie": cookie_string,
        "Origin": "https://pump.fun",
        "Referer": "https://pump.fun/",
        "Sec-Ch-Ua": "\"Microsoft Edge\";v=\"125\", \"Chromium\";v=\"125\", \"Not.A/Brand\";v=\"24\"",
        "Sec-Ch-Ua-Mobile": "?0",
        "Sec-Ch-Ua-Platform": "\"Windows\"",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-site",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36 Edg/125.0.0.0"
    }

    data = {
        "text": text,
        "mint": mint,
        "token": token
    }

    try:
        response = requests.post('https://frontend-api.pump.fun/replies', json=data, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as err:
        print(f"Error response: {err.response.json() if err.response else err}")
        raise

# Post positive messages to live chat
def post_positive_message(driver, wallet):
    positive_messages = [
        "Great stream!",
        "This is awesome!",
        "Loving the content!",
        "Keep up the good work!",
        "This livestream is amazing!"
    ]
    message = random.choice(positive_messages)

    # Extract necessary cookies from the browser
    cookies = driver.get_cookies()
    additional_cookies = {cookie['name']: cookie['value'] for cookie in cookies}

    try:
        result = send_post_request(message, "your_mint_value", "your_token_value", wallet['auth_token'], additional_cookies)
        logging.info(f"Sent message: {message} - {wallet['publicKey']} with result: {result}")
    except Exception as e:
        logging.error(f"Failed to post message with wallet {wallet['publicKey']}: {e}")

# Open a browser, click "join livestream," and post messages
def browse_page(browser_id, retries=3):
    global wallet_index
    user_agent = random.choice(USER_AGENTS)
    driver = initialize_driver(user_agent)

    try:
        logging.info(f"Browser {browser_id} with user-agent {user_agent} opened.")
        driver.get(target_url)
        logging.info(f"Browser {browser_id}: Navigated to {target_url}.")

        # Random sleep to simulate natural browsing behavior
        time.sleep(random.uniform(2, 5))

        # Wait for the "join livestream" button and click it
        join_button = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "div.inline-flex.items-center.justify-center.whitespace-nowrap.rounded-md.font-medium.ring-offset-white.transition-colors.focus-visible\\:outline-none.focus-visible\\:ring-2.focus-visible\\:ring-slate-950.focus-visible\\:ring-offset-2.disabled\\:pointer-events-none.disabled\\:opacity-50.dark\\:ring-offset-slate-950.dark\\:focus-visible\\:ring-slate-300.dark\\:hover\\:bg-slate-800.dark\\:hover\\:text-slate-50.h-10.px-4.py-2.-mt-5.text-2xl.text-slate-50.hover\\:font-bold.hover\\:bg-transparent.hover\\:text-slate-50.cursor-pointer"))
        )
        join_button.click()
        logging.info(f"Browser {browser_id}: Clicked 'join livestream' button.")

        # Wait for the livestream page to load by checking for a video element
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "video.lk-participant-media-video"))
        )
        logging.info(f"Browser {browser_id}: Livestream loaded successfully.")

        # Post a positive message in the chat using a Solana wallet
        if wallet_index < len(wallets):
            post_positive_message(driver, wallets[wallet_index])
            wallet_index += 1
        else:
            logging.warning(f"Browser {browser_id}: No more wallets available to post messages.")

        logging.info(f"Browser {browser_id}: Staying on livestream for 1 hour.")
        time.sleep(3600)  # Stay on the page for 1 hour

        logging.info(f"Browser {browser_id}: Completed session.")

    except Exception as e:
        logging.error(f"Browser {browser_id}: Error during browsing: {e}")
        if retries > 0:
            logging.info(f"Browser {browser_id}: Retrying... ({retries} attempts left)")
            driver.quit()
            browse_page(browser_id, retries - 1)  # Retry the process
    finally:
        driver.quit()
        logging.info(f"Browser {browser_id}: Closed.")

# Open multiple browser instances
def open_multiple_browsers(num_browsers=3):
    with ThreadPoolExecutor(max_workers=num_browsers) as executor:
        for i in range(num_browsers):
            executor.submit(browse_page, i + 1)

# Start the browsing session
open_multiple_browsers()
